## リザバーコンピューティング
参考：\
「リザバーコンピューティング 時系列パターン認識のための高速機械学習の理論とハードウェア 田中剛平・中根了昌・廣瀬明[共著]」\
https://www.morikita.co.jp/books/mid/085531

### ・中間層のネットワークの作成
・作成の流れ\
1, 全結合のネットワーク(0と1の正方行列)を作成する。\
2, 指定の密度になるまでランダムに枝切りをする\
→ スパースなネットワークの完成

In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random

### 1, 全結合のネットワークを作成する
引数：ノード数 node\
戻り値：全結合の行列 w

In [4]:
def AllConnect(node):
    
    w = np.zeros((node,node))
    for i in range(node):
        for j in range(node):
            if i != j:
                w[i,j]=1
                    
    return w

#### 確認する

In [5]:
node = 5
AllConnect(node)

array([[0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1.],
       [1., 1., 0., 1., 1.],
       [1., 1., 1., 0., 1.],
       [1., 1., 1., 1., 0.]])

### 2, 枝切りする
引数1：ネットワークを表す結合行列 w\
引数2：密度 density\
戻り値：指定の密度の結合行列 w

### 枝切りのために必要となるステップ
1, 引数として渡された結合行列の結合数を把握する。\
2, 指定の密度にするために残すべき枝の数を把握する。

### 1, 引数として渡された結合行列の結合数を把握する
### reserch_graph(w)
引数：結合行列\
戻り値：結合がある座標をリストにしたもの\
(例)ノード1とノード5に結合がある場合は (0,4)を返す

In [7]:
def reserch_graph(w):
    
    access_list = []
    n1, n2 = w.nonzero()
        
    for nn1, nn2 in zip(n1, n2):
        l1 = [nn1,nn2]
        access_list.append(l1)
    
    return access_list

#### 確認する
ノード数=5, 全結合の場合で確認する。\
結合数が5×(5-1)=20になっていればOK

In [9]:
node = 5
w_all = AllConnect(node) #全結合のネットワークを作成
access_list = reserch_graph(w_all) #結合を確認
len(access_list) == 20

True

In [13]:
access_list

[[0, 1],
 [0, 2],
 [0, 3],
 [0, 4],
 [1, 0],
 [1, 2],
 [1, 3],
 [1, 4],
 [2, 0],
 [2, 1],
 [2, 3],
 [2, 4],
 [3, 0],
 [3, 1],
 [3, 2],
 [3, 4],
 [4, 0],
 [4, 1],
 [4, 2],
 [4, 3]]

### 2, 指定の密度にするために残すべき枝の数を把握する。
### check_pathnum
引数1：ノード数 node\
引数2：結合密度 density\
戻り値：必要な結合数(枝の数)\
(例) ノード数=5, 結合密度=0.1の場合、5×(5-1)×0.1=2本を返す。

In [14]:
def check_pathnum(node,density):
    m = node*(node-1)
    path_num = int(m*density)
    
    return path_num

#### 確認する
(例) ノード数=5, 結合密度=0.1の場合、5×(5-1)×0.1=2本になればOK

In [16]:
node = 5
density = 0.1
check_pathnum(node,density)==2

True

### スパースネットワークの作成
### make_sparse_graph
check_pathnumとreserch_graphを使って作成する\

引数1：結合行列 w\
引数2：ノード数 node\
引数3：結合密度 density\
戻り値：スパースな結合行列

In [21]:
def make_sparse_graph(w,node,density):
    
    path_num = check_pathnum(node,density) #残すべき結合数の計算
    access_list = reserch_graph(w) #結合状態の確認
    length = len(access_list)
        
    access_num = np.arange(length)
    random.shuffle(access_num)
    
    for i in range( length-path_num ):
        
        num = access_num[i]
    
        remove_1 = access_list[num][0]
        remove_2 = access_list[num][1]
    
        w[remove_1,remove_2] = 0
    
    return w

#### 確認する
テスト1：\
「全結合 → node=5, density=0.5」の時でテストする。\
結合数=10(5×4×0.5)になっていればOK

In [23]:
node = 5
density = 0.5
w_all = AllConnect(node) #全結合のネットワークを作成
w_sparse = make_sparse_graph(w_all,node,density)
w_sparse

array([[0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0.]])

テスト2：\
「node=5,density=0.5 → node=5, density=0.1」の時でテストする。\
結合数=2(5×4×0.1)になっていればOK

In [25]:
density = 0.1
w_sparse_af = make_sparse_graph(w_sparse,node,density)
w_sparse_af

array([[0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])